# Welcome to the Musical Gestures Toolbox (Python) - Tutorial/Documentation

## Video visualisation
Videos can be watched as they are, but they can also be used to develop new visualisations to be used for analysis. The aim of creating such alternate displays from video recordings is to uncover features, structures and similarities within the material itself, and in relation to, for example, score material. Three useful visualisation techniques here are motion images, motion history images and motiongrams.

MGT can generate both dynamic and static visualizations, as well as some quantitative data:

- dynamic visualisations (video files)
    - motion video
    - motion history video
- static visualisations (images)
    - motion average image
    - motiongrams
    - videograms
- motion data (csv files)
    - quantity of motion
    - centroid of motion
    - area of motion

In the following we will try this ourselves, and look at the different types.

## Using Google Colab
In case you are using this notebook in Google Colab, there are a few additional steps to make sure everything works. First, let's download and install the module:

In [ ]:
!pip install musicalgestures

If you are prompted to restart the runtime, do so by clicking the button in the previous cell. This is necessary if the required version of IPython is newer than the one that is preinstalled on Colab.

Now we should install a relatively new version of FFmpeg (again, we need a bit newer one than what comes with Colab). First we add a repository that has the newer version, and then we download and install it. 

In [ ]:
!add-apt-repository -y ppa:jonathonf/ffmpeg-4
!apt install --upgrade ffmpeg

## Import

Now let's import the `musicalgestures`.

In [ ]:
import musicalgestures

To make your first steps a bit easier, we packaged a couple of example videos into `musicalgestures`. We will use these throughout this notebook to demonstrate the different functions in the package. To simplify our future code, let's save these into variables:

In [ ]:
# an example video with a dencer
dance = musicalgestures.examples.dance
# another example video with a pianist
pianist = musicalgestures.examples.pianist

## The MgVideo

### Simple video import

Now we create our mg (musical gestures) object. 

In [ ]:
mg = musicalgestures.MgVideo(dance)

You can watch your video with calling the `show()` method. There are two modes to choose from. The default `'windowed'` mode will open a video player as a separate window. In `'notebook'` mode the video is embedded into the notebook. Since only mp4, webm and ogg file formats are compatible with browsers, `show` will automatically convert your video to mp4 if necessary. If you use the `musicalgestures` package in Colab, note that `show` will use notebook-mode in any case.

In [ ]:
# mg.show() # this would open in a separate window
mg.show(mode="notebook") # this embeds the video in the notebook

## Preprocessing modules
### Trimming
When creating the object you can also already apply some preprocessing. For example you can trim the duration of the video like this:

In [ ]:
# the numbers used for starttime and endtime represent time from the video's timeline in seconds
mg = musicalgestures.MgVideo(dance, starttime=5, endtime=15)
mg.show() # view the result

This will create the file *dance_trim.avi* in the same directory.

### Skipping
In order to save time, skipping every other frame, or more, in the analysis can give you a faster analysis while still getting an idea of the motion. You can for example set this by adding `skip=2`, to skip two frames before including a frame in the analysis, then skipping two again. 

In [ ]:
mg = musicalgestures.MgVideo(dance, starttime=5, endtime=15, skip=2)
mg.show()

This will create the file *dance_trim_skip.avi* in the same directory. Notice how the added suffixes at the end of the file's name can inform you about the processes the material went through. You probably also noticed, that an additional "_0" suffix has been appended to the trimmed result. By default all the functions that produce files in the package will silently increment output file names to avoid overwriting existing files. 

### It's a chain
It is also worth to note that the preprocessing modules work as a chain (- more on that below). In this case that means we first load the video file, then trim its start to 5s and its end to 15s. Then we take the resulting *dance_trim.avi* and discard 2 out of every 3 frames. (Keeping the 1st, skipping 2nd and 3rd, keeping the 4th, skipping 5th and 6th, and so on...) The resulting file of this process is *dance_trim_skip.avi*.

### Rotating

Sometimes source videos are recorded with a slightly off horizon, or with the camera mounted sideways, therefore it is desirable to rotate the video by a few (or more) degrees. We can do this by simply specifying the angle we want to rotate with for the `rotate` parameter of our MgVideo:

In [ ]:
# rotate by 90 degrees
mg = musicalgestures.MgVideo(dance, starttime=5, endtime=15, skip=3, rotate=90)
mg.show()

In [ ]:
# or just a little bit...
mg = musicalgestures.MgVideo(dance, starttime=5, endtime=15, skip=3, rotate=5.31)
mg.show()

Again, the resulting filename *dance_trim_skip_rot.avi* will inform us about the chain of processes *dance.avi* went through.

### Adjusting contrast and brightness
During preprocessing you can also add (or remove) some contrast and brightness of your video.

Let's crank up the contrast and brighten up our *dance.avi*!

In [ ]:
mg = musicalgestures.MgVideo(dance, starttime=5, endtime=15, skip=3, rotate=90, contrast=100, brightness=20)
mg.show()

The resulting file is now called *dance_trim_skip_rot_cb.avi*.

### Cropping

If the video frame has big areas with no motion occuring, a lot of time could be saved if only the area with motion was used in the analysis. One useful tool developed for the pre-analysis is the crop = 'auto' input, which automatically finds the area with significant motion in the input video. The movement occuring has to be above a low threshold, as to not include irrelevant background motion from shadows, dust etc. Another mode is crop = 'manual', where you can manually mark a rectangle around your area of interest.

#### Automatic cropping

In [ ]:
mg = musicalgestures.MgVideo(dance, starttime=5, endtime=15, skip=3, rotate=90, contrast=100, brightness=20, crop='auto')
mg.show()

#### Manual cropping

In [ ]:
mg = musicalgestures.MgVideo(dance, starttime=5, endtime=15, skip=3, rotate=90, contrast=100, brightness=20, crop='manual')
mg.show()

The resulting file is now called *dance_trim_skip_rot_cb_crop.avi*.

### Grayscale mode

So far all of our work preserved the color space of the source video. At the final preprocessing stage we can also choose to convert the video to grayscale with specifying `color=False` (by default `color=True`). This will not only result in a grayscale version of our source video, but also informs all future processes to work in grayscale mode. The technical benefit of this can be slightly shorter processing times at most processes, since in grayscale mode we process a single color channel per frame (instead of three channels). Try this:

In [ ]:
mg = musicalgestures.MgVideo(dance, starttime=5, endtime=15, skip=3, rotate=90, contrast=100, brightness=20, crop='auto', color=False)
mg.show()

The resulting file is now called *dance_trim_skip_rot_cb_crop_gray.avi*.

### Summary of preprocessing modules
As we have seen, we can optionally apply six types of preprocessing to the video we load into our MgVideo, they are (in order of execution):

- trim: Trim contents of the video based on `starttime` and `endtime`.
- skip: Skip every n frames, where n is determined by `skip`.
- rotate: Rotate the video by an angle determined by `rotate`.
- cb: Adjust contrast and brightness of the video, where `contrast` and `brightness` are the level of adjustment in percentages (meaning `contrast=0` will not apply any change). both values range from `-100` to `100`.
- crop: Crop frames in video. If `crop='auto'` the module will attempt to find the area of motion, if `crop='manual'` we can draw the cropping rectangle over the first frame.
- grayscale: Convert the video to grayscale with specifying `color=False`. This will also cause all further processes called on the MgVideo to function in grayscale mode.


### Keep everything
Notice that although we can optionally apply up to four preprocessing modules to our source video, normally we only keep the final result. If you would like to keep the results of all modules, set `keep_all=True`.

In [ ]:
mg = musicalgestures.MgVideo(dance, starttime=5, endtime=15, skip=3, rotate=90, contrast=100, brightness=20, crop='auto', color=False, keep_all=True)

This will output six new video files:
- *dance_trim.avi*
- *dance_trim_skip.avi*
- *dance_trim_skip_rot.avi*
- *dance_trim_skip_rot_cb.avi*
- *dance_trim_skip_rot_cb_crop.avi*
- *dance_trim_skip_rot_cb_crop_gray.avi*

## Processes

In the following we will take a look at several functions to further process our videos. These include:
- `motion()`: The most frequently used function, generates a *_motion* video, horizontal and vertical motiongrams, and plots about the centroid and quantity of motion found in the video.
- `motiongrams()`: A fast shortcut to `motion()` that only outputs the motiongrams.
- `motiondata()`: A shortcut to only output the motion data as a csv file.
- `motionvideo()`: A fast shortcut to only render the *_motion* video.
- `videograms()`: Outputs the videograms.
- `history()`: Renders a *_history* video by layering the last n frames on the current frame for each frame in the video.
- `average()`: Renders an *_average* image of all frames in the video.
- `flow.sparse()`: Renders a *_sparse* optical flow video.
- `flow.dense()`: Renders a *_dense* optical flow video.
- `pose()`: Renders a *_pose* human pose estimation video, and optionally outputs the pose data as a csv file.
- `audio.waveform()`: Renders a figure showing the waveform of the video/audio file.
- `audio.spectrogram()`: Renders a figure showing the mel-scaled spectrogram of the video/audio file.
- `audio.descriptors()`: Renders a figure of plots showing spectral/loudness descriptors, including RMS energy, spectral flatness, centroid, bandwidth, rolloff of the video/audio file.
- `audio.tempogram()`: Renders a figure with a plots of onset strength and tempogram of the video/audio file.

### Motion analysis

By calling the `motion()` function, we will generate a number of files from the input video, in the same location as the source file.

These include:
- *<input_filename>_motion.avi*: The motion video that is used as the source for the rest of the analysis.
- *<input_filename>_mgx.png*: A horizontal motiongram.
- *<input_filename>_mgy.png*: A vertical motiongram.
- *<input_filename>_motion_com_qom.png*: An image file with plots of centroid and quantity of motion

We will examine each of these in a little more detail.

In [ ]:
mg = musicalgestures.MgVideo(dance, starttime=5, endtime=15, skip=1, contrast=100, brightness=20, crop='auto')
mg.motion()

We can now look at the results with using the `key` parameter of `show()`.

In [ ]:
mg.show(key='motion') # show the motion video of the preprocessed input, in this case 'dance_trim_skip_cb_crop_motion.avi'

In [ ]:
mg.show(key='mgx') # show the horizontal motiongram, here 'dance_trim_skip_cb_crop_mgx.png'

In [ ]:
mg.show(key='mgy') # show the vertical motiongram

In [ ]:
mg.show(key='plot') # show the image of the two plots ('Centroid of motion' and 'Quantity of motion') also shown at the end of motion()

Alternatively we can display the images right here in our notebook:

In [ ]:
mg.show(key='mgx', mode='notebook') # default is mode='windowed'

In [ ]:
mg.show(key='mgy', mode='notebook')

In [ ]:
mg.show(key='plot', mode='notebook')

You can also configure `motion()` to output only the files you need:

In [ ]:
mg = musicalgestures.MgVideo(dance, starttime=5, endtime=15, skip=1, contrast=100, brightness=20, crop='auto')

In [ ]:
# without plot
mg.motion(save_plot=False)

In [ ]:
# without plot and data
mg.motion(save_plot=False, save_data=False)

In [ ]:
# without plot, data and motiongrams (so only the video)
mg.motion(save_plot=False, save_data=False, save_motiongrams=False)

In [ ]:
# without video, plot and motiongrams (so only the data)
mg.motion(save_plot=False, save_video=False, save_motiongrams=False)

What if you only want _one_ kind of output? Then you should choose from the shortcuts below. `motionvideo()` actually renders the video much faster than `motion()` since it does not have to extract centroid and quantity of motion information.

In [ ]:
mg.motionvideo() # only the _motion video (super fast)
mg.motiongrams() # only the motiongrams (as slow as `motion()`)
mg.motionplots() # only the motion plots (as slow as `motion()`)
mg.motiondata() # only the motion data (as slow as `motion()`)

When it comes to the motion data, you can choose from several different formats:

In [ ]:
mg = musicalgestures.MgVideo(dance, starttime=5, endtime=15, skip=1, contrast=100, brightness=20, crop='auto')

In [ ]:
# save only the data as .csv (default)
mg.motiondata()

In [ ]:
# save only the data as .tsv
mg.motiondata(data_format="tsv")

In [ ]:
# save only the data as .txt
mg.motiondata(data_format="txt")

In [ ]:
# saving in multiple formats if data_format is a list
mg.motiondata(data_format=["txt", "csv"])

#### Filtering types
If you think there is too much noise in the output images or video, you may choose to use some other filter settings.

Filtertypes available are:

- `Regular` turns all values below `thresh` to 0.
- `Binary` turns all values below `thresh` to 0, above `thresh` to 1.
- `Blob` removes individual pixels with erosion method.

Try this:

In [ ]:
mg.motion(filtertype='Blob').show(mode='notebook')

#### Effects of filtering

Finding the right `thresh`old value is crucial for accurate motion extraction. Let's see a few examples.

First we import the example video.

In [ ]:
mg = musicalgestures.MgVideo(dance, starttime=5, endtime=10, skip=0, contrast=100, brightness=20)

First we can try to run without any threshold. This will result in a result in which much of the background noise will be visible, including traces of keyframes if the video file has been compressed.

In [ ]:
mg.motiongrams(thresh=0.0).show(mode='notebook')

Adding just a little bit of thresholding (0.02 here) will drastically improve the final result.

In [ ]:
mg.motiongrams(thresh=0.02).show(mode='notebook')

The standard threshold value (0.1) generally works well for many types of videos.

In [ ]:
mg.motiongrams(thresh=0.1).show(mode='notebook')

A more extreme value (for example 0.5) will remove quite a lot of the content, but may be useful in some cases with very noisy videos.

In [ ]:
mg.motiongrams(thresh=0.5).show(mode='notebook')

As the above examples have shown, choosing the thresholding value is important for the final output result. While it often works to use the default value (0.1), you may improve the result by testing different thresholds.

### Videograms
Creating motiongrams can give you a quick visual overview about the motion in your video. In many cases it can be equally informative to get the videograms of your video. These images can give you a more complete overview of the whole scene, and perhaps a useful comparision to the motiongrams, since it does not remove static (non-moving) parts of the image.

In [ ]:
mg.videograms().show(mode='notebook')

### History tracking
As we have seen above, `motion()` is useful if you want to remove the still content of your video, only keeping what is different in subsequent frames. Sometimes it is also useful to visualize changes between frames in a different way: layering the last n frames on top of the current one as a video delay. With `history()` you can achieve this, optionally setting the `history length` to the number of past frames you want to see on the current frame (ie. the length of the delay).

Try this:

In [ ]:
mg.history()
mg.show(key='history')

By default `history_length=10`. Let's increase it to 20!

In [ ]:
mg.history(history_length=20).show()

### Motion history

To expressively visualize the trajectory of a moving content in a video, you can apply the history process on a motion video. You can do this by chaining `motion()` into `history()`. (More about chaining below!)

In [ ]:
mg.motion().history().show()

### Average image
You can also summarize the content of a video by showing the average of all frames in a single image.

In [ ]:
mg.average().show(mode='notebook')

### Optical flow
It is also possible to track the direction certain points - or all points - move in a video, this is called 'optical flow'. It has two types: the *sparse optical flow*, which is for tracking a small (sparse) set of points, visualized with an overlay of dots and lines drawing the trajectory of the chosen points as they move in the video.  

In [ ]:
mg.flow.sparse().show()

Note that sparse optical flow usually works well with slow and continuous movements, where the points to be tracked are not occluded by other objects throughout the course of motion.
Where spare optical flow becomes less reliable, *dense optical flow* often yields more robust results. In dense optical flow the analysis attempts to track the movement of each pixel (or more precisely groups of pixels), colorcoding them with a unique color for each unique direction.

In [ ]:
mg.flow.dense().show()

Sparse optical flow can get confused by too fast movement (ie. too big distance between the locations of a tracked point between two consequtive frames), so it is typically advised not to have a too high `skip` value in the preprocessing stage for it to work properly.
Dense optical flow on the other hand has issues with very slow movement, which sometimes gets below the treshold of what is considered 'a movement' resulting in a blinking video, where the more-or-less idle moments are rendered completely black. If your source video contains such moments, you can try setting `skip_empty=True`, which will discard all the (completely) black frames, eliminating the binking. 

In [ ]:
mg.flow.dense(skip_empty=True).show()

### Pose estimation
This module uses a more advanced type of computer vision, that involves a deep neural network trained by a huge dataset of images of people (courtesy of [OpenPose](https://github.com/CMU-Perceptual-Computing-Lab/openpose)!) and tries to estimate their skeleton by tracking a set of "keypoints", which are joints on the body - for example "Head", "Left Shoulder", "Right Knee", etc. After the module runs you can take a look at the *_pose.csv* dataset, that contains the normalized XY pixel coordinates of each keypoint, and you can visualize the result with drawing a skeleton overlay over your video. You can choose from three trained models: the MPI (which is trained on the Multi-Person Dataset), the COCO model (trained on the COCO Dataset) or the BODY25 model. The module also supports GPU-acceleration, so if you have compiled openCV with CuDNN support, you can make the - otherwise rather slow - inference process run over 10 times faster!

#### The models
Since both models are quite large (~200MB each) they do not "ship" with the musicalgestures package, but we do include some convenience bash/batch scripts do download them on the fly if you need them. If the `pose()` module cannot find the model you asked for it will offer you to download it.

#### Downsampling
Running inference on large neural networks to process every pixel of every frame of your video is quite a costly operation. There is a trick however to reduce the load and this is downsampling your input image. Often times a large part of the frame is redundant and the posture of the person in the video can easily be understood on a lower resolution image as well. Downsampling can greatly speed up `pose()`, but of course it can also make its estimation less accurate if overused. The default value we use in `pose()` is `downsampling_factor=2` which produces a video with one-fourth of its original resolution before feeding it to the network.

#### Confidence threshold
The networks are not always equally confident about their guesses. Sometimes (especially with heavy downsampling) they can identify other objects in your scene as either of the keypoints of the human body we wish to track. Filtering out inconfident guesses can remove a lot of noise from the prediction. `pose()` has a normalized `threshold` parameter that is set to `0.1`. This means the network has to be at least 10% sure about its guess for us to take that prediction into account.

Below you can find a simple example of `pose()` in action. For more info check out the [documentation](https://github.com/fourMs/MGT-python/blob/master/musicalgestures/documentation/_pose.md).

In [ ]:
mg.pose(downsampling_factor=1, threshold=0.05, model='mpi', device='gpu').show(mode='notebook')

## Audio

The Musical Gestures Toolbox offers several tools to analyze the audio track of videos or audio files. These are implemented both as class methods for `MgObject` and standalone functions. At their core they use the `librosa` package for audio analysis and the `matplotlib` package for showing the analysis as figures, which you can also save as images (*.png* files). In order to make working with these figures simpler and more flexible we use our own `MgFigure` class as a data structure. To find out how you can combine several `MgFigure`-s check the section **Figures and Plotting** below.

### Waveforms

A [waveform](https://en.wikipedia.org/wiki/Waveform) is a plot of audio samples (y axis) against time (x axis). It can provide a basic decsription of the audio content. Here is how you can create a waveform of an audio track/file:

In [ ]:
mg = musicalgestures.MgVideo(pianist)
mg.audio.waveform()

### Spectrograms

A [spectrogram](https://en.wikipedia.org/wiki/Spectrogram) is a plot of frequency spectrum (y axis) against time (x axis). It can provide a much more descriptive representation of audio content than a waveform (which is in a way the sum of all frequencies with respect to their phases). Here is how you can create a spectrogram of an audio track/file:

In [ ]:
mg = musicalgestures.MgVideo(pianist)
mg.audio.spectrogram()

This has created a figure showing the [mel-scaled](https://en.wikipedia.org/wiki/Mel_scale) spectrogram, and rendered *dance_spectrogram.png* in the same folder where our input video, *dance.avi* resides.

### Tempograms

Tempograms attempt to use the same technique (called [Fast Fourier Transform](https://en.wikipedia.org/wiki/Fast_Fourier_transform)) as spectrograms to estimate musical tempo of the audio. In `tempogram()` we analyze the onsets and their strengths throughout the audio track, and then estimate the global tempo based on those. Here is how to use it:

In [ ]:
mg = musicalgestures.MgVideo(pianist)
mg.audio.tempogram()

Estimating musical tempo meaningfully is a tricky thing, as it is often a function of not just onsets (beats), but the underlying harmonic structure as well. `tempogram()` only relies on onsets to make its estimation, which can in some cases identify the most common beat frequency as the "tempo" (rather than the _actual_ musical tempo).

### Descriptors

Additionally to spectrograms and tempogams you can also get a collection of audio descriptors via `descriptors()`. This collection includes:
- RMS energy,
- spectral flatness, 
- spectral centroid, 
- spectral bandwidth, 
- and spectral rolloff.

RMS energy is often used to get a perceived loudness of the audio signal. Spectral flatness indicates how _flat_ the graph of the spectrum is at a given point in time. Noisier signals are more flat than harmonic ones. The spectral centroid shows the centroid of the spectrum, spectral bandwidth marks the the frequency range where power drops by less than half (at most −3 dB). Spectral rolloff is the frequency below which a specified percentage of the total spectral energy, e.g. 85%, lies. `descriptors()` draws two rolloff lines: one at 99% of the energy, and another at 1%.

In [ ]:
mg = musicalgestures.MgVideo(pianist)
mg.audio.descriptors()

## Figures and Plotting

The Musical Gestures Toolbox includes several tools to extract data from audio-visual content, and many of these tools output figures (or images) to visualize this time-varying data. In this section we take a closer look on how we can customize and combine figures and images from the toolbox.

### Titles

By default the figures rendered by the toolbox automatically get the title of the source file we analyzed. We can also change this by providing a title as an argument to the function or method we are using. Here are some examples:

In [ ]:
# source video as an MgObject
mg = musicalgestures.MgVideo(pianist)

# motion plots
motionplots = mg.motionplots(title='Liszt - Mephisto Waltz No. 1 - motion')

# spectrogram
spectrogram = mg.audio.spectrogram(title='Liszt - Mephisto Waltz No. 1 - spectrogram')

# tempogram
tempogram = mg.audio.tempogram(title='Liszt - Mephisto Waltz No. 1 - tempogram')

# descriptors
descriptors = mg.audio.descriptors(title='Liszt - Mephisto Waltz No. 1 - descriptors')

## Combining figures and images

In this section we take a look at how we can compose time-aligned figures easily within the Musical Gestures Toolbox.

### Helper data structures: MgFigure and MgList

First let us take a look at the data structures which help us through the composition. 

### MgFigure

As we have `MgVideo` for videos and `MgImage` for images, we also use a dedicated data structure for `matplotlib` figures: the `MgFigure`. Normally, you don't need a custom data-structure to just deal with a figure, but `MgFigure` offers an organized, comfortable way to represent the type of the figure and its data, so that we can reuse it in other figures. First of all, it implements the `show()` method which is used across the `musicalgestures` package to show the content of an object. In the case of `MgFigure` this will show the internal `matplotlib.pyplot.figure` object:

In [ ]:
spectrogram.show()

Fun fact: as `show()` is implemented only for the sake of consistency here, you can achieve the same by referring to the internal figure of the `MgFigure` object directly:

In [ ]:
spectrogram.figure # same as spectrogram.show()

What is more important is that each `MgFigure` has a `figure_type` attribute. This is what you see when you `print` (or in a notebook such as this, evaluate) them:

In [ ]:
spectrogram

Each `MgFigure` object has a `data` attribute, where we store all the related data to be able to recreate the figure elsewhere. You never really have to interact with this attribute directly, unless you want to look under the hood:

In [ ]:
print(spectrogram.data.keys()) # see what kind of entries we have in our spectrogram figure
print(spectrogram.data['length']) # see the length (in seconds) of the source file

You can also get the rendered image file corresponding to the `MgFigure` object. As an exercise, here is how you can make an `MgImage` of this image file and show it embedded in this notebook:

In [ ]:
musicalgestures.MgImage(spectrogram.image).show(mode='notebook')

Another attribute of `MgFigure` is called `layers`. We will get back to this in a bit, for now let's just say that when an `MgFigure` object is in fact a composition of other `MgFigure`, `MgImage` or `MgList` objects, we have access to all those in the `layers` of the "top-level" `MgFigure`.

### MgList

Another versatile tool in our hands is `MgList`. It works more-or-less as an ordinary list, and it is specifically designed for working with objects of the `musicalgestures` package. Here is an example how to use it:

In [ ]:
from musicalgestures._mglist import MgList
liszt_list = MgList(spectrogram, tempogram)
liszt_list

`MgList` also implements many of the `list` feaures you already know:

In [ ]:
# how many objects are there?
print(f'There are {len(liszt_list)} objects in this MgList!')

# which one is the 2nd?
print(f'The second object is a(n) {liszt_list[1]}.')

# change the 2nd element to the descriptors figure instead:
liszt_list[1] = descriptors
print(f'The second object is now {liszt_list[1]}.') # check results

# add the tempogram figure to the list:
liszt_list += tempogram
print(f'Now there are {len(liszt_list)} objects in this MgList. These are:') # check results
for element in liszt_list:
    print(element)

# fun fact: videograms() returns an MgList with the horizontal and vertical videograms (as MgImages)
videograms = musicalgestures.MgVideo(pianist).videograms()

# MgList.show() will call show() on all its objects in a succession
videograms.show()

# add two MgLists
everything = videograms + liszt_list
print('everything:', everything)

### Composing figures with MgList

One of the most useful methods of `MgList` is `as_figure()`. It allows you to conveniently compose a stack of plots, time-aligned, and with a vertical order you specify. Here is an example:

In [ ]:
fig_everything = everything.as_figure(title='Liszt - Mephisto Waltz No. 1')

## Chaining

So far our workflow consisted of the following steps:
- 1. Creating an MgVideo which loads a video file and optionally applies some preprocessing to it.
- 2. Calling a process on the MgVideo.
- 3. Viewing the result.

Something like this:

In [ ]:
mg = musicalgestures.MgVideo(dance, starttime=5, endtime=15, skip=3)
mg.motion()
mg.show(key='motion')

This is convenient if you want to apply several different processes on the same input video.

In [ ]:
mg = musicalgestures.MgVideo(dance, starttime=5, endtime=15, skip=3)
mg.motion()
mg.history()
mg.average()

The Musical Gestures Toolbox also offers an alternative workflow in case you want to apply a proccess on the result of a previous process. Although `show()` is not really a process (ie. it does not yield a file as a result) it can provide a good example of the use of chaining:

In [ ]:
# this...
mg.motion().show()

In [ ]:
# ...is the equivalent of this!
mg.motion()
mg.show(key='motion')

It also works with images:

In [ ]:
mg.average().show()

But chaining can go further than this. How about reading (and preprocessing) a video, rendering its motion video, the motion history and the average of the motion history, with showing the *_motion_history_average.png* at the end - all as a one-liner?!

In [ ]:
musicalgestures.MgVideo(dance, skip=4, crop='auto').motion().history().average().show(mode='notebook')

In [ ]:
# equivalent without chaining
mg = musicalgestures.MgVideo(dance, skip=4, crop='auto')
mm = mg.motion()
mh = mm.history()
mh.average()
mh.show(key='average', mode='notebook')

Some other examples:

In [ ]:
# rendering and viewing the motion video 
musicalgestures.MgVideo(dance, skip=4).motion().show(mode='notebook')

In [ ]:
# rendering the motion video, the motion history video, and viewing the latter
musicalgestures.MgVideo(dance, skip=3).motion().history(normalize=True).show(mode='notebook')

In [ ]:
# rendering the motion video, the motion average image, and viewing the latter
musicalgestures.MgVideo(dance, skip=15).motion().average().show(mode='notebook')